#### Para vencer al Mago Goma (juego de Radio Metro)

In [1]:
import pyphen
import numpy as np

In [2]:
dic = pyphen.Pyphen(lang='es')

In [3]:
class Monosilaba(Exception):
    """Excepcion lanzada cuando la palabra es monosilaba"""
    pass

class Palabra_Usada(Exception):
    """Excepcion lanzada cuando se repite una palabra"""
    pass

def normalizar(palabra):
    """Quita acentos y pone todo en minuscula """
    return quitar_acentos(palabra).lower()

def es_valida(dic, palabra, ultima_silaba):
    """Valida si la ultima silaba de una palabra es la primera silaba de otra palabra"""
    f, l = separar_en_silabas(dic, palabra)
        
    return f == ultima_silaba 

def palabra_usada(palabra, pablas_usadas):
    if normalizar(palabra) in palabras_usadas:
        raise Palabra_Usada
    return False
        
        
    
def quitar_acentos(silabas):
    """Reemplaza los acentos en las vocales de las silabas por su equivalentes sin acentos"""
    reemplazo = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
    }

    for k, v in reemplazo.items():
        silabas = silabas.replace(k, v)
    return silabas
    

def separar_en_silabas(dic, palabra):
    """Separa una palabra en silabas y devuelve la primera y ultima silibas de la misma o 
    lanza la excepción Monosilaba si la palabra no se puede seguir dividiendo"""
    if not es_monosilaba(dic, palabra):
        silabas = quitar_acentos(dic.inserted(palabra))
        first = silabas.split('-')[0]
        last = silabas.split('-')[-1]
        return first, last
    else:
        raise Monosilaba

def es_monosilaba(dic, palabra):
    """Devuelve True si la palabra es monosilaba y False sino"""
    return not '-' in dic.inserted(palabra) 

def agrupar_palabras_por_primera_silaba(palabras):
    """Crea un diccionario donde las claves son las primeras silabas de las palabras y 
    los valores son las listas de palabras que comienzan con esa silaba"""
    #agrupar palabras por la primera silaba para agilizar busqueda
    dic_silabas = {}
    for w in es_words:
        if not es_monosilaba(dic, w):
            first, _ = separar_en_silabas(dic, w)
            if not first in dic_silabas.keys():
                dic_silabas.update({first: [w]})
            else:
                dic_silabas[first].append(w)
    return dic_silabas

#### Obtenemos una lista de palabras en español

In [4]:
with open('palabras_espanol.txt') as f:
    es_words = f.read().splitlines()

In [5]:
dic_silabas = agrupar_palabras_por_primera_silaba(es_words)

#### Jugamos CPU vs CPU!

In [6]:
np.random.seed(17)
try:
    jugador = False
    palabra = 'lorca'
    palabras_usadas = []
    for i in range(50):
        palabras_usadas.append(normalizar(palabra))
        print(f" *** Round {i+1} *** ")
        f, l = separar_en_silabas(dic, palabra)
        options = np.array(dic_silabas[l])
        respuesta = np.random.choice(options)
        palabra = respuesta
        print(f"Jugador n°{int(jugador)+1}, {palabra}")
        jugador = not jugador
        palabra_usada(palabra, palabras_usadas)
except Monosilaba:
    print(f"Perdiste! :( {palabra} es monosilaba")
except KeyError:
    print(f"Jugador n°{int(jugador)+1} perdió: No tiene palabras para {palabra}")
except Palabra_Usada:
    print(f"Perdiste! :( {palabra} ya fue usada")

 *** Round 1 *** 
Jugador n°1, cábala
 *** Round 2 *** 
Jugador n°2, layador
 *** Round 3 *** 
Jugador n°1, dorsoventral
 *** Round 4 *** 
Jugador n°2 perdió: No tiene palabras para dorsoventral


#### Jugamos User VS CPU!

In [7]:
np.random.seed(17)
palabras_usadas = []
try:    
    for i in range(50):
        print(f"User: ")
        palabra = input()
        desconfio = False
        
        if palabra == "--des--":
            palabra = respuesta
            desconfio = True
            l, f = separar_en_silabas(dic, palabra)
        
        if not desconfio:
            palabra_usada(palabra, palabras_usadas)
            palabras_usadas.append(normalizar(palabra))

        if i != 0 and not es_valida(dic, palabra, l):
            print(f"Perdiste! :( '{palabra}' no es valida para la silaba '{l}' ")
            break
            
        print(f" *** Round {i+1} *** ")    
        f, l = separar_en_silabas(dic, palabra)
        options = np.array(dic_silabas[l])
        respuesta = np.random.choice(options)
        
        palabra = respuesta
        f, l = separar_en_silabas(dic, palabra)
        print(f"Jugador CPU, {palabra}")
        
        if desconfio and es_valida(dic, palabra, f) and not palabra_usada(respuesta, palabras_usadas):
            print(f"Perdiste!")
            break
            
        palabra_usada(respuesta, palabras_usadas)
        palabras_usadas.append(normalizar(respuesta))
        
        
except Monosilaba:
    print(f"Perdiste! :( {palabra} es monosilaba")
except KeyError:
    print(f"Jugador CPU perdió: No tiene palabras para {palabra}")
except Palabra_Usada:
    print(f"Perdiste! :( {palabra} ya fue usada")
    
print(f"******* Puntaje Obtenido: {i} *******")

User: 


 televisor


 *** Round 1 *** 
Jugador CPU, sordilla
User: 


 llave


 *** Round 2 *** 
Jugador CPU, verriondez
User: 


 --des--


 *** Round 3 *** 
Jugador CPU, dezmeño
Perdiste!
******* Puntaje Obtenido: 2 *******


#### TODO: 
- refactorizar
- ~~Llevar un registro del puntaje (No es puntaje, sino la cantidad de ROUNDS que te la bancas)~~
- ~~User vs CPU~~
- Llevar un registro de palabras usadas
- Agregar un diccionario
- ~~Implementar "desconfio"~~
- Implementar speech2text
- Conseguir un corpus más amplio de palabras en español